# BRDF Representation

## Mitsuba "Deep Dive"

In [1]:
import drjit as dr
import mitsuba as mi
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

print(mi.variants())
mi.set_variant("llvm_ad_rgb_double")

['scalar_mono', 'scalar_mono_double', 'scalar_rgb', 'scalar_spectral', 'llvm_ad_rgb', 'llvm_ad_rgb_double']


In [2]:
WHITECAP=0
GLINT=1
UNDERLIGHT=2
TOTAL=3

channels = [WHITECAP, GLINT, UNDERLIGHT, TOTAL]

In [3]:
res = 25
azim_vs = np.linspace(0, 2 * np.pi, 2 * res)
zen_vs = np.linspace(0, np.deg2rad(89), res)

wavelength = 2.2
incoming = 15
solar_azimuth = 0
wind_speed = 10
wind_direction = np.deg2rad(0)
n_real = 1.333
n_imag = 0.0
chlorinity = 19
pigmentation = 0.3

In [4]:
def sph_to_eucl(theta, phi):
    st, ct = dr.sincos(theta)
    sp, cp = dr.sincos(phi)
    return mi.Vector3f(cp * st, sp * st, ct)

: 

In [ ]:
brdf_values = None
brdf_foam = None
brdf_glint = None
brdf_underlight = None

# Create a dummy surface interaction to use for the evaluation of the BSDF
si = dr.zeros(mi.SurfaceInteraction3f)

# Specify an incident direction with X degrees
si.wi = sph_to_eucl(dr.deg2rad(incoming), dr.deg2rad(solar_azimuth))

# Set the surface normal to point upwards
si.n = mi.Vector3f(0, 0, 1)

# Create grid in spherical coordinates and map it into a sphere
res = 500
zeniths_o, azimuths_o = dr.meshgrid(
    dr.linspace(mi.Float, np.deg2rad(1), np.deg2rad(89),     res),
    dr.linspace(mi.Float, 0, 2 * dr.pi, 2 * res)
)
wo = sph_to_eucl(zeniths_o, azimuths_o)

for channel in channels:
    bsdf = mi.load_dict({
        'type': 'oceanic_legacy',
        'channel': channel,
        'visual_type': 0,
        'wavelength': wavelength,
        'wind_speed': wind_speed,
        'wind_direction': wind_direction,
        'chlorinity': chlorinity,
        'pigmentation': pigmentation,
        'shininess': 50,
    })
    
    #result = bsdf.eval(mi.BSDFContext(), si, wo)
    if channel == WHITECAP:
        brdf_foam = bsdf.eval(mi.BSDFContext(), si, wo)
    elif channel == GLINT:
        brdf_glint = bsdf.eval(mi.BSDFContext(), si, wo)
    elif channel == UNDERLIGHT:
        brdf_underlight = bsdf.eval(mi.BSDFContext(), si, wo)
    elif channel == TOTAL:
        brdf_values = bsdf.eval(mi.BSDFContext(), si, wo)

In [ ]:
brdf_np = np.array(brdf_values)
foam_np = np.array(brdf_foam)
glint_np = np.array(brdf_glint)
underlight_np = np.array(brdf_underlight)

In [ ]:
brdf_data = brdf_np[:,0]
foam_data = foam_np[:,0]
glint_data = glint_np[:,0]
underlight_data = underlight_np[:,0]

In [ ]:
# Extract red channel of BRDF values and reshape into 2D grid
brdf_data = brdf_data.reshape(2 * res, res).T
foam_data_vis = foam_data.reshape(2 * res, res).T
glint_data_vis = glint_data.reshape(2 * res, res).T
underlight_data_vis = underlight_data.reshape(2 * res, res).T

dimensional_data = brdf_data.T
foam_buffer = foam_data_vis.T
glint_buffer = glint_data_vis.T
underlight_buffer = underlight_data_vis.T

width = len(dimensional_data)
height = len(dimensional_data[0])

foam_data = np.zeros((width, height))
glint_data = np.zeros((width, height))
underlight_data = np.zeros((width, height))

for x in range(width):
    for y in range(height):
        f = foam_buffer[x][y]
        g = glint_buffer[x][y]
        u = underlight_buffer[x][y]
        total_val = dimensional_data[x][y]
        foam_comp = f / total_val
        glint_comp = g / total_val
        underlight_comp = u / total_val

        foam_data[x][y] = foam_comp
        glint_data[x][y] = glint_comp
        underlight_data[x][y] = underlight_comp

In [ ]:
# Plot values for spherical coordinates
fig, ax = plt.subplots(figsize=(8, 4))
visual_set = [foam_data_vis, glint_data_vis, underlight_data_vis, brdf_data]
channel = 3
im = ax.imshow(visual_set[channel], interpolation='spline36', extent=[0, 2 * np.pi, np.pi / 2, 0], cmap='turbo')

# Name the axes
plt.xlabel("Outgoing Azimuth")
plt.ylabel("Outgoing Zenith")

# Add the title
degrees = np.degrees(incoming)

# Round to 2 decimal places
degrees = round(degrees, 2)

# Create x_axis tick labels in degrees
x_ticks = np.linspace(0, 2 * np.pi, 5)
x_labels = np.degrees(x_ticks)
x_labels = [round(label, 2) for label in x_labels]
plt.xticks(x_ticks, x_labels)

# Create y_axis tick labels in degrees
y_ticks = np.linspace(0, np.pi / 2, 3)
y_labels = np.degrees(y_ticks)
y_labels = [round(label, 2) for label in y_labels]
plt.yticks(y_ticks, y_labels)

# Add Title
ax.set_title(f'BRDF | Incoming Zenith: {incoming}°')

# Add color bar with same height as the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.25)
plt.colorbar(im, cax=cax)
plt.show()

In [ ]:
# Min/Max values
min_val = np.min(brdf_data)
max_val = np.max(brdf_data)
mean_val = np.mean(brdf_data)
mean_exluding_zeros = np.mean(brdf_data[brdf_data != 0])
zeros = np.count_nonzero(brdf_data == 0)

print(f"Min: {min_val}, Max: {max_val}, Mean: {mean_val}, Mean excluding zeros: {mean_exluding_zeros}")
print(f"Zeros: {zeros}, Total: {res * 2 * res}, Percentage: {zeros / (res * 2 * res) * 100}%")

## Converting to Polar representation of the BRDF

In [ ]:
# Combine zeniths, azimuths and BRDF values into a single array
zeniths = np.array(zeniths_o)
azimuths = np.array(azimuths_o)
brdf_data_new = np.array(brdf_data).T.flatten()
data = np.stack([zeniths, azimuths, brdf_data_new], axis=1)

# Remove rows where the zenith (first element) is greater than 90 degrees
data = data[data[:, 0] <= np.pi / 2]

# Extract the new values
zeniths_t = data[:, 0]
azimuths_t = data[:, 1]
brdf_data_t = data[:, 2]

### Simple Polar Plot

In [ ]:
def visualize():
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.scatter(azimuths_t, zeniths_t, c=brdf_data_t, cmap='turbo', s=1)
    ax.grid(False)
    ax.set_yticklabels([])
    cbar = fig.colorbar(im, ax=ax, orientation='vertical')
    ax.set_ylim(0, np.pi / 2)
    plt.show()

#visualize()

In [ ]:
# Visualize component values
slice_azimuth = np.deg2rad(180)

# Find the index of the slice azimuth
t = slice_azimuth / (2 * np.pi)
slice_index = int(t * 2 * res)

# Extract the slice
slice_foam_data = foam_buffer[slice_index, :]
slice_glint_data = glint_buffer[slice_index, :]
slice_underlight_data = underlight_buffer[slice_index, :]

# Plot the slice
fig, ax = plt.subplots()

# Plot the slice by rescaling the x-axis between 0 and 90 degrees
x = np.linspace(0, np.pi / 2, res)

# X to degrees
x = np.degrees(x)

ax.plot(x, slice_foam_data, label='Foam')
ax.plot(x, slice_glint_data, label='Glint')
ax.plot(x, slice_underlight_data, label='Underlight')

ax.set_xlabel('Outgoing Zenith')
ax.set_ylabel('Component Value')
ax.set_title(f'Slice of BRDF')
plt.show()

In [ ]:
# Visualize a slice of the BRDF
slice_azimuth = np.deg2rad(180)

# Find the index of the slice azimuth
t = slice_azimuth / (2 * np.pi)
slice_index = int(t * 2 * res)

# Extract the slice
slice_data = dimensional_data[slice_index, :]

# Plot the slice
fig, ax = plt.subplots()

# Plot the slice by rescaling the x-axis between 0 and 90 degrees
x = np.linspace(0, np.pi / 2, res)

# X to degrees
x = np.degrees(x)

ax.plot(x, slice_data, label='Slice')
ax.set_xlabel('Outgoing Zenith')
ax.set_ylabel('BRDF Value')
ax.set_title(f'Slice of BRDF')
plt.show()


In [ ]:
# Component Diagnostic
print(f'Foam:')
print(f'    Min: {np.min(foam_data)}')
print(f'    Max: {np.max(foam_data)}')
print(f'    Mean: {np.mean(foam_data)}')

print(f'Glint:')
print(f'    Min: {np.min(glint_data)}')
print(f'    Max: {np.max(glint_data)}')
print(f'    Mean: {np.mean(glint_data)}')

print(f'Underlight:')
print(f'    Min: {np.min(underlight_data)}')
print(f'    Max: {np.max(underlight_data)}')
print(f'    Mean: {np.mean(underlight_data)}')

In [ ]:
# Visualize a slice of the components of the BRDF
slice_azimuth = np.deg2rad(180)

# Find the index of the slice azimuth
t = slice_azimuth / (2 * np.pi)
slice_index = int(t * 2 * res)

# Extract the slice
slice_foam_data = foam_data[slice_index, :]
slice_glint_data = glint_data[slice_index, :]
slice_underlight_data = underlight_data[slice_index, :]

# Plot the slice
fig, ax = plt.subplots()

# Plot the slice by rescaling the x-axis between 0 and 90 degrees
x = np.linspace(0, np.pi / 2, res)

# X to degrees
x = np.degrees(x)

ax.plot(x, slice_foam_data, label='Foam')
ax.plot(x, slice_glint_data, label='Glint')
ax.plot(x, slice_underlight_data, label='Underlight')

ax.set_xlabel('Outgoing Zenith')
ax.set_ylabel('BRDF Component Share')
ax.set_title(f'Slice of BRDF')

plt.legend()

plt.show()